In [1]:
import numpy as np

from src.item_category.base import ItemCategorySelectionPool
from src.order_profile.base import OrderProfile
from src.sampling.distributions import Distribution

cat_selection_pools = {
    "turn_rates": ItemCategorySelectionPool(
        item_category_id=1,
        likelihood_lower_bound=0.9,
        likelihood_upper_bound=1,
        category_quantity_distribution=Distribution(
            distribution_type="normal",
            lower_bound=1,
            upper_bound=1,
            int_or_float="int"
        ),
        n_services=10,
        lower_price_bound=1000,
        upper_price_bound=5000,
        price_distribution_type="longtail",
        int_or_float_price="float",
        likelihood_range=(1, 1),
        quantity_distribution_types=["normal"],
        quantity_upper_bound=(1, 2),
        quantity_int_or_float="int",
        variant_distribution_type=None,
        variant_upper_bound=None,
        variant_lower_bound=None,
    ),
    "labour_rates": ItemCategorySelectionPool(
        item_category_id=2,
        likelihood_lower_bound=0.2,
        likelihood_upper_bound=0.5,
        category_quantity_distribution=Distribution(
            distribution_type="normal",
            lower_bound=0,
            upper_bound=3,
            int_or_float="int"
        ),
        n_services=15,
        lower_price_bound=30,
        upper_price_bound=200,
        price_distribution_type="longtail",
        int_or_float_price="float",
        likelihood_range=(0.1, 0.6),
        quantity_distribution_types=["longtail"],
        quantity_upper_bound=(1, 5),
        quantity_int_or_float="int",
        variant_distribution_type="longtail",
        variant_upper_bound=1,
        variant_lower_bound=0,
    ),
    "plane_services": ItemCategorySelectionPool(
        item_category_id=3,
        likelihood_lower_bound=0.2,
        likelihood_upper_bound=0.5,
        category_quantity_distribution=Distribution(
            distribution_type="normal",
            lower_bound=0,
            upper_bound=2,
            int_or_float="int"
        ),
        n_services=15,
        lower_price_bound=200,
        upper_price_bound=500,
        price_distribution_type="longtail",
        int_or_float_price="float",
        likelihood_range=(0.1, 1),
        quantity_distribution_types=["longtail"],
        quantity_upper_bound=(1, 2),
        quantity_int_or_float="int",
        variant_distribution_type=None,
        variant_upper_bound=None,
        variant_lower_bound=None,
    ),
    "additional_services": ItemCategorySelectionPool(
        item_category_id=3,
        likelihood_lower_bound=0.4,
        likelihood_upper_bound=0.7,
        category_quantity_distribution=Distribution(
            distribution_type="normal",
            lower_bound=0,
            upper_bound=2,
            int_or_float="int"
        ),
        n_services=15,
        lower_price_bound=100,
        upper_price_bound=500,
        price_distribution_type="longtail",
        int_or_float_price="float",
        likelihood_range=(0.1, 1),
        quantity_distribution_types=["longtail"],
        quantity_upper_bound=(1, 2),
        quantity_int_or_float="int",
        variant_distribution_type=None,
        variant_upper_bound=None,
        variant_lower_bound=None,
    ),
}

In [2]:
profiles = [
    OrderProfile(
        customer_id=customer_id,
        item_categories=[cat_selection_pools[key].sample_items(n_samples=np.random.randint(3, 5)) for key in cat_selection_pools.keys()],
    ) for customer_id in range(1, 51)
]

In [4]:
import pandas as pd

n_cycles = 10
rounds_per_cycle = 50
orders: list[pd.DataFrame] = []

for cycle in range(n_cycles):
    for r in range(rounds_per_cycle):
        for profile in profiles:
            increased = False
            if profile.increase_viable():
                if np.random.choice([True, False], p=[0.8, 0.2]):
                    profile.modify_prices_random(
                        factor=round(((np.random.rand() * 2) - 1) / 10, 2)
                    )
                    profile.reset_increase()
                    increased = True
            order = profile.sample()
            order["contract_ammendment"] = increased
            order["round"] = f"{cycle}_{r}"
            orders.append(order)

orders_df = pd.concat(orders)

In [5]:
orders_df

,service_id,price,quantity,variant,item_category_id,conditions,customer_id,contract_ammendment,round
0,1_1,1235.620000,1.0,None,1.0,[],1.0,False,0_0
0,2_14,353.530000,1.0,0,2.0,[],1.0,False,0_0
0,3_11,815.820000,1.0,None,3.0,[3_probability],1.0,False,0_0
0,1_5,6564.350000,1.0,None,1.0,[],2.0,False,0_0
0,3_2,114.200000,1.0,None,3.0,[],2.0,False,0_0
...,...,...,...,...,...,...,...,...,...
0,3_9,863.790000,1.0,None,3.0,[3_probability],48.0,False,9_49
0,1_2,166.809297,1.0,None,1.0,[],49.0,False,9_49
0,2_12,111.940000,1.0,0,2.0,[],49.0,False,9_49
0,3_3,2518.690000,1.0,None,3.0,[3_probability],49.0,False,9_49
